## 加载数据 

In [1]:
import torch
import torch.nn as nn

In [2]:
import torchvision as tv
import torchvision.transforms as transforms
import torch.utils.data

In [3]:
show = tv.transforms.ToPILImage()  # 将Tensor转化为Image
# 定义对数据的预处理
transform = transforms.Compose([transforms.ToTensor(),  # 转为Tensor
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化
                                ])
# 训练集
train_set = tv.datasets.CIFAR10(root='/home/new1006/ljh/data/cifar-10-python',
                                train=True,
                                download=True,
                                transform=transform)

train_loader =torch.utils.data.DataLoader(train_set,
                                          batch_size=64,  # 4
                                          shuffle=True,
                                          drop_last=True,
                                          num_workers=4)

# 测试集
test_set = tv.datasets.CIFAR10(root='/home/new1006/ljh/data/cifar-10-python',
                               train=False,
                               download=True,
                               transform=transform)
  
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=64,  # 4
                                          shuffle=False,
                                          drop_last=True)
#                                           num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 模型结构

In [6]:
train_set

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: E:\Data\dataSet\cifar-10-python
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
class Net(nn.Module):
    """输入为3通道的网络"""
    def __init__(self):
        super(Net, self).__init__()
        cba1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(32),
                nn.LeakyReLU()
                )
        cba2 = nn.Sequential(
                nn.Conv2d(32, 32, 3, 2, 1),
                nn.BatchNorm2d(32),
                nn.LeakyReLU()
                )
        cba3 = nn.Sequential(
                nn.Conv2d(32, 64, 3, 1, 1),
                nn.BatchNorm2d(64),
                nn.LeakyReLU()
                )
        cba4 = nn.Sequential(
                nn.Conv2d(64, 64, 3, 2, 1),
                nn.BatchNorm2d(64),
                nn.LeakyReLU()
                )
        
#         self.moudlelist_features = nn.ModuleList([cba1, cba2, cba3, cba4])  # 方法一
        
        features = [cba1, cba2, cba3, cba4]  # 方法二
        self.features = nn.Sequential(*features)
        
        self.classifier = nn.Sequential(
                    nn.Linear(4096, 128),
                    nn.BatchNorm1d(128),
                    nn.LeakyReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(128, 10),
                    nn.Softmax(dim=1)
                    ) 
        
    def forward(self, x):
#         x = self.moudlelist_features(x)  # 会报错，Module_List没有实现forward()方法
#         for model in self.moudlelist_features:  # 方法一
#             x = model(x)
        
        x = self.features(x)  # 方法二
        
        x = x.view(x.size()[0], -1)  # 按照Batch形式排列成[Batch, C*W*H]
#         x = nn.Linear(in_features=x.size()[-1], out_features=128)(x)  # 将模型放到cuda上时，在forward()中初始化层会报错；cpu上没问题
#         x = nn.functional.linear(input=x, weight=torch.randn(128, x.size()[-1]), bias=torch.randn(128))
        x = self.classifier(x)
        return x

In [8]:
net = Net()

### 将模型数据放入cuda 

In [9]:
next(net.parameters()).device  # 判断模型位置

device(type='cpu')

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda')
net = net.to(device)  # 整个式子等价与net.cuda()

In [11]:
device

device(type='cuda')

In [12]:
next(net.parameters()).device

device(type='cuda', index=0)

In [13]:
input_data = torch.randn(64, 3, 32, 32)   # 判断数据位置

In [14]:
input_data.device

device(type='cpu')

In [15]:
input_data = input_data.cuda()

In [16]:
input_data.device

device(type='cuda', index=0)

### 使用torchsummay.summary展示层结构 

In [17]:
from torchsummary import summary

In [18]:
for name, p in net.named_parameters():
    print(name, p.size())

features.0.0.weight torch.Size([32, 3, 3, 3])
features.0.0.bias torch.Size([32])
features.0.1.weight torch.Size([32])
features.0.1.bias torch.Size([32])
features.1.0.weight torch.Size([32, 32, 3, 3])
features.1.0.bias torch.Size([32])
features.1.1.weight torch.Size([32])
features.1.1.bias torch.Size([32])
features.2.0.weight torch.Size([64, 32, 3, 3])
features.2.0.bias torch.Size([64])
features.2.1.weight torch.Size([64])
features.2.1.bias torch.Size([64])
features.3.0.weight torch.Size([64, 64, 3, 3])
features.3.0.bias torch.Size([64])
features.3.1.weight torch.Size([64])
features.3.1.bias torch.Size([64])
classifier.0.weight torch.Size([128, 4096])
classifier.0.bias torch.Size([128])
classifier.1.weight torch.Size([128])
classifier.1.bias torch.Size([128])
classifier.4.weight torch.Size([10, 128])
classifier.4.bias torch.Size([10])


In [19]:
summary(model = net, input_size = (3, 32, 32), batch_size = 64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 32, 32, 32]             896
       BatchNorm2d-2           [64, 32, 32, 32]              64
         LeakyReLU-3           [64, 32, 32, 32]               0
            Conv2d-4           [64, 32, 16, 16]           9,248
       BatchNorm2d-5           [64, 32, 16, 16]              64
         LeakyReLU-6           [64, 32, 16, 16]               0
            Conv2d-7           [64, 64, 16, 16]          18,496
       BatchNorm2d-8           [64, 64, 16, 16]             128
         LeakyReLU-9           [64, 64, 16, 16]               0
           Conv2d-10             [64, 64, 8, 8]          36,928
      BatchNorm2d-11             [64, 64, 8, 8]             128
        LeakyReLU-12             [64, 64, 8, 8]               0
           Linear-13                  [64, 128]         524,416
      BatchNorm1d-14                  [

In [20]:
summary(model = net, input_size = (3, 32, 32), batch_size = 4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [4, 32, 32, 32]             896
       BatchNorm2d-2            [4, 32, 32, 32]              64
         LeakyReLU-3            [4, 32, 32, 32]               0
            Conv2d-4            [4, 32, 16, 16]           9,248
       BatchNorm2d-5            [4, 32, 16, 16]              64
         LeakyReLU-6            [4, 32, 16, 16]               0
            Conv2d-7            [4, 64, 16, 16]          18,496
       BatchNorm2d-8            [4, 64, 16, 16]             128
         LeakyReLU-9            [4, 64, 16, 16]               0
           Conv2d-10              [4, 64, 8, 8]          36,928
      BatchNorm2d-11              [4, 64, 8, 8]             128
        LeakyReLU-12              [4, 64, 8, 8]               0
           Linear-13                   [4, 128]         524,416
      BatchNorm1d-14                   

## 训练 

In [21]:
import torch.optim as optim
from torch.autograd import Variable

In [22]:
optimizer = optim.Adam(params=net.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [23]:
import time

In [27]:
running_loss = 0
start_time = time.time()
for epoch in range(10):
    for i, data in enumerate(train_loader):
        images, labels = data
        images, labels = Variable(images), Variable(labels)
        if torch.cuda.is_available():
            net.cuda()
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 打印log信息
       
        running_loss += loss.data
        if i % 260 == 259:  # 每2000个batch打印一次训练信息
            print('[%d %5d] loss: %.3f  time: %.3fs' % (epoch+1, i+1, running_loss/260, time.time()-start_time))
            running_loss = 0
            start_time = time.time()
print('Finish Training')

[1   260] loss: 1.587  time: 2.827s
[1   520] loss: 1.590  time: 2.700s
[1   780] loss: 1.599  time: 2.670s
[2   260] loss: 1.587  time: 2.867s
[2   520] loss: 1.593  time: 2.570s
[2   780] loss: 1.592  time: 2.604s
[3   260] loss: 1.584  time: 2.942s
[3   520] loss: 1.581  time: 2.692s
[3   780] loss: 1.590  time: 2.687s
[4   260] loss: 1.582  time: 2.905s
[4   520] loss: 1.583  time: 2.639s
[4   780] loss: 1.589  time: 2.639s
[5   260] loss: 1.580  time: 2.893s
[5   520] loss: 1.580  time: 2.624s
[5   780] loss: 1.579  time: 2.609s
[6   260] loss: 1.577  time: 2.853s
[6   520] loss: 1.573  time: 2.644s
[6   780] loss: 1.580  time: 2.637s
[7   260] loss: 1.573  time: 2.901s
[7   520] loss: 1.570  time: 2.653s
[7   780] loss: 1.579  time: 2.665s
[8   260] loss: 1.569  time: 2.978s
[8   520] loss: 1.573  time: 2.660s
[8   780] loss: 1.570  time: 2.610s
[9   260] loss: 1.568  time: 2.911s
[9   520] loss: 1.568  time: 2.708s
[9   780] loss: 1.568  time: 2.685s
[10   260] loss: 1.567  time

In [29]:
# 整个测试集上效果
total = 0  # 总共的图片数
correct = 0  # 正确的图片数
for images, labels in test_loader:
    outputs = net(Variable(images).cuda())
    _, predicted = torch.max(outputs, 1)
    total += predicted.size()[0]  # 加4
    correct += (predicted == labels.cuda()).sum()
print('correct: %d, total: %d' %(correct, total))
print('整个测试集中的准确率为 %.2f%%' % (100 * correct / total))

correct: 7370, total: 9984
整个测试集中的准确率为 73.82%
